In [1]:
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pandas as pd
import random
import pandas as pd
import numpy as np

In [3]:
def get_row_id(row):
    return str(row.frame) + "-" + row.type + "-" + str(row.landmark_index)

def duplicate_vals(pf, values: np.array, iter):
    frames_to_dup = pf.loc[pf.frame.isin(values)].copy()
    frames_to_dup.frame += (frames_to_dup.frame-round(frames_to_dup.frame))*0.001 + 0.01*iter
    
    frames_to_dup.row_id = frames_to_dup.apply(get_row_id, axis=1)

    pf = pd.concat([pf, frames_to_dup], ignore_index=True).sort_values('frame')

    return pf

def remove_vals(pf, values: np.array):
    values = np.sort(values)

    for val in values:
        pf = pf.loc[pf.frame != val]

    return pf


def transform_data(pf, frame_amt_goal, iter=1):

    frame_nums = pf.frame.unique()
    frame_diff = abs(frame_amt_goal - len(frame_nums))
    operation = frame_amt_goal > len(frame_nums)

    values_to_operate = np.array([])

    if frame_diff%2 == 1:
        central_point = frame_nums[int(len(frame_nums)/2)]
        values_to_operate = np.append(values_to_operate, [central_point])
        frame_nums = np.delete(frame_nums, [int(len(frame_nums)/2)])
        frame_diff -= 1

    if frame_diff != 0:
        step_val = len(frame_nums)/frame_diff
        step_val = 1 if step_val < 1 else step_val
        
        loop_cnt = len(frame_nums) if frame_diff > len(frame_nums) else frame_diff
        values_to_operate = np.append(values_to_operate, frame_nums[[int(i*step_val) for i in range(0, loop_cnt)]])
    else:
        loop_cnt = 0

    if operation:
        pf = duplicate_vals(pf, values_to_operate, iter)
    else:
        pf = remove_vals(pf, values_to_operate)

    if frame_diff - loop_cnt != 0:
        pf = transform_data(pf, frame_amt_goal, iter+1)

    return pf

def populate_table(pf, video_data):
    frame_num = 0

    for frame in pf.frame.unique():
        x_vals = list(pf['x'].loc[pf.frame==frame])
        y_vals = list(pf['y'].loc[pf.frame==frame])
        z_vals = list(pf['z'].loc[pf.frame==frame])

        video_data[f'{frame_num}x'] = x_vals
        video_data[f'{frame_num}y'] = y_vals
        video_data[f'{frame_num}z'] = z_vals
        
        frame_num += 1

    return video_data

def create_data_table(pf):
    col_labels = ['type','landmark_index']

    for i in range(len(pf.frame.unique())):
        col_labels.append(f'{i}x')
        col_labels.append(f'{i}y')
        col_labels.append(f'{i}z')

    landmarks = []
    types = []

    for i in pf.type.unique():
        for j in pf.landmark_index.loc[pf.type==i].unique():
            landmarks.append(j)
            types.append(i)

    data = {col: [0.0] * len(types) for col in col_labels}
    data['type'] = types
    data['landmark_index'] = landmarks

    video_data = pd.DataFrame(columns=col_labels, data=data)
    video_data = populate_table(pf, video_data)

    return video_data

def drop_empty_rows(pf):
    pf = pf.drop(pf.loc[(pf.x == 0) & (pf.y == 0) & (pf.z == 0)].index, axis=0)
    return pf

In [14]:
folders = ["alligator/", "flower/", "kiss/", "listen/", "orange/"]
aggregated_pf = pd.DataFrame()

le = LabelEncoder()
le.fit(['right_hand','left_hand','pose'])

file_id = 0
unique_ids = random.sample(range(1000, 10000), 50*len(folders))

for folder in folders:
    path = "./asl-kaggle/averaged_by_labels/"+folder
    # path = "./asl-kaggle/by_labels/"+folder
    parquets = [f for f in listdir(path) if isfile(join(path, f))]

    for parquet in parquets[:50]:
        parquet_path = path+parquet
        pf = pd.read_parquet(parquet_path)
        
        pf = pf.drop(pf.loc[pf.type=="face"].index)
        pf = pf.fillna(0)

        pf = transform_data(pf, 30)
        pf = drop_empty_rows(pf)
        video_data = create_data_table(pf)
        video_data['type'] = le.transform(video_data['type'])
        video_data['label'] = folder
        video_data['file_id'] = file_id

        aggregated_pf = pd.concat([aggregated_pf, video_data])

        file_id += 1

In [ ]:
# print(tf.__version__)
# # To make the results reproducible, set the random seed value.
# tf.random.set_seed(22)

In [7]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.Variable(tf.math.reduce_mean(x, axis=0))
    self.std = tf.Variable(tf.math.reduce_std(x, axis=0)) + 0.001

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

In [15]:
scores = []

# Iterate x amount of times to get an average accuracy score
for i in range(20):
    # Retrieve file ids
    ids = aggregated_pf['file_id'].unique()
    # Calculate the numerical value of 75% of ids
    num_elements_to_select = int(0.75 * len(ids))
    # Choose at random 75% of ids from dataset for train test
    train_values = np.random.choice(ids, size=num_elements_to_select, replace=False)
    # Remaining ids assign to test set
    test_values = ids[~np.isin(ids, train_values)]

    # Retrieve data from dataset where file_id is in list of train values
    train_dataset = aggregated_pf.loc[(aggregated_pf.file_id.isin(train_values))]
    # Remove the label from train_dataset and assign to x_test
    # Retrieve labels and assign to y_train
    x_train, y_train = train_dataset.drop(['label'], axis=1), train_dataset['label']

    # Covnert dataframe to tensor
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)

    # Normalize train set
    norm_x = Normalize(x_train)
    x_train_norm = norm_x.norm(x_train)

    # Fit train data
    classif = LogisticRegression(random_state=0, max_iter=10000).fit(x_train_norm, y_train)
    # classif = RandomForestClassifier().fit(pca_x_train, y_train)
    # classif = SVC(gamma='auto').fit(pca_x_train, y_train)

    # Initialize list of actual labels and predicitions
    y_test = []
    y_preds = []

    # Loop over file_ids in test_values
    for id in test_values:
        # Append actual label of data at file id
        y_test.append(aggregated_pf.label.loc[(aggregated_pf.file_id == id)].unique()[0])

        # Retrieve data for provided file id and drop label
        x_test = aggregated_pf.loc[(aggregated_pf.file_id == id)].drop(['label'], axis=1)
        # Convert test data to tensor
        x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
        # Normalize test data
        x_test_norm =  norm_x.norm(x_test)

        # Predict labels for each row in dataset
        pred = classif.predict(x_test_norm)
        # Initialize a counter object to find most common label
        counter = Counter(pred)
        # Find most common label in predictions
        pred_label = counter.most_common(1)[0][0]
        # Append predicted label to list of predictions
        y_preds.append(pred_label)

    # Calculate accuracy for test set and append to list of scores
    scores.append(accuracy_score(y_preds, y_test))

# Find the average score for all iterations
print(np.mean(scores))

0.9785714285714286
